In [10]:
import pandas as pd
df = pd.read_csv("Titanic-dataset/train.csv",encoding='utf8')
df.groupby("Survived").count()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Survived,,,,,,,,,,,
0,549,549,549,549,424,549,549,549,549,68,549
1,342,342,342,342,290,342,342,342,342,136,340


In [16]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 采用中位数填充缺失的“Age”

In [22]:
df1 = df.fillna({"Age":df["Age"].median()})
df1.groupby("Survived").count()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Survived,,,,,,,,,,,
0,549,549,549,549,549,549,549,549,549,68,549
1,342,342,342,342,342,342,342,342,342,136,340


In [23]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 线性拟合预测缺失值

In [120]:
from sklearn.linear_model import LinearRegression
df2 = df[["Survived","Pclass","Age","SibSp","SibSp","Fare"]]
df2.head()
df_na = df2[df2["Age"].isna()]
df_na = df_na.reset_index()
df_no_na = df2[-df2["Age"].isna()]
df_no_na = df_no_na.reset_index()

### 创建模型
linreg = LinearRegression()
linreg.fit(df_no_na[["Survived","Pclass","SibSp","SibSp","Fare"]].values, df_no_na["Age"].values)

### 预测Age为空的数据
na_predict = linreg.predict(df_na[["Survived","Pclass","SibSp","SibSp","Fare"]])
print(na_predict[na_predict<0])
print(na_predict[na_predict>0][:10])

## 用模型预测出来的NaN值，有些事负数，而年龄至少>=1，因此对于是负数的设置为1
def transformage(age):
    if(age<0):
        return 1
    else:
        return age
df_na_predict_age = [transformage(age) for age in linreg.predict(df_na[["Survived","Pclass","SibSp","SibSp","Fare"]].values)]

## 用预测出的值替换掉NaN值
df_na['Age'] = pd.Series(df_na_predict_age)
df_na.info()
df_na.head()

[-6.12312288 -6.12312288 -6.12312288 -6.12312288 -6.12312288 -6.12312288
 -6.12312288]
[28.92041514 30.0841245  22.0263679  28.94226788 22.01477619 28.93038203
 31.63323157 22.01706547 22.02629348 28.93038203]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 7 columns):
index       177 non-null int64
Survived    177 non-null int64
Pclass      177 non-null int64
Age         177 non-null float64
SibSp       177 non-null int64
SibSp       177 non-null int64
Fare        177 non-null float64
dtypes: float64(2), int64(5)
memory usage: 9.8 KB


,index,Survived,Pclass,Age,SibSp,SibSp,Fare
0,5,0,3,28.920415,0,0,8.4583
1,17,1,2,30.084125,0,0,13.0000
2,19,1,3,22.026368,0,0,7.2250
3,26,0,3,28.942268,0,0,7.2250
4,28,1,3,22.014776,0,0,7.8792


In [121]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [125]:
df.groupby("Cabin").count()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
Cabin,,,,,,,,,,,
A10,1,1,1,1,1,1,1,1,1,1,1
A14,1,1,1,1,1,0,1,1,1,1,1
A16,1,1,1,1,1,1,1,1,1,1,1
A19,1,1,1,1,1,0,1,1,1,1,1
A20,1,1,1,1,1,1,1,1,1,1,1
A23,1,1,1,1,1,1,1,1,1,1,1
A24,1,1,1,1,1,1,1,1,1,1,1
A26,1,1,1,1,1,1,1,1,1,1,1
A31,1,1,1,1,1,1,1,1,1,1,1
